# Ball Knower v1.0 - Deterministic Spread Model Demo

This notebook demonstrates the Ball Knower v1.0 model - a simple, deterministic spread predictor combining nfelo and Substack power ratings.

**Model Philosophy:**
- No machine learning, no hyperparameter tuning
- Pure Python + pandas with interpretable components
- Pre-game features only (no leakage)
- Weighted combination of proven rating systems

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from ball_knower.models.v1_0 import build_week_lines

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Load Week 11 (2025) Predictions

The `build_week_lines()` function:
- Loads team ratings via the unified loader
- Loads the game schedule
- Joins ratings for home/away teams
- Calculates spread components
- Combines into final `bk_line` prediction

In [ ]:
# Generate predictions for Week 11
predictions = build_week_lines(season=2025, week=11)

print(f"Generated predictions for {len(predictions)} games in Week 11, 2025\n")
predictions.head()

## 2. Preview Predictions vs Vegas Lines

Let's look at our predictions compared to Vegas spreads.

**Spread Convention:**
- Negative spread = home team favored
- Positive spread = away team favored

In [ ]:
# Display key columns
display_cols = [
    "game_id",
    "gameday",
    "away_team",
    "home_team",
    "bk_line",
    "vegas_line",
]

pred_display = predictions[display_cols].copy()

# Calculate difference (positive means we're more optimistic about home team)
pred_display["bk_vs_vegas"] = pred_display["vegas_line"] - pred_display["bk_line"]

# Round for readability
pred_display["bk_line"] = pred_display["bk_line"].round(1)
pred_display["bk_vs_vegas"] = pred_display["bk_vs_vegas"].round(1)

print("\n=== Ball Knower v1.0 Predictions - Week 11, 2025 ===")
print(pred_display.to_string(index=False))

## 3. Component Breakdown

Let's examine the individual components that make up our predictions:
- **nfelo_diff**: nfelo rating difference (away - home + HFA)
- **substack_power_diff**: Substack overall rating difference
- **epa_off_diff**: Offensive EPA per play difference
- **epa_def_diff**: Defensive EPA per play difference

In [ ]:
# Component analysis
component_cols = [
    "away_team",
    "home_team",
    "nfelo_diff",
    "substack_power_diff",
    "epa_off_diff",
    "epa_def_diff",
    "bk_line",
]

# Filter to columns that exist
component_cols = [c for c in component_cols if c in predictions.columns]
component_display = predictions[component_cols].copy()

# Round for readability
for col in component_display.columns:
    if col not in ["away_team", "home_team"]:
        component_display[col] = component_display[col].round(3)

print("\n=== Component Breakdown ===")
print(component_display.to_string(index=False))

## 4. Summary Statistics

Let's look at some basic statistics about our predictions.

In [ ]:
# Calculate statistics
print("\n=== Summary Statistics ===")
print(f"\nNumber of games: {len(predictions)}")
print(f"\nBK Line statistics:")
print(predictions["bk_line"].describe())

if "vegas_line" in predictions.columns:
    # Calculate correlation and mean absolute difference
    valid_vegas = predictions[predictions["vegas_line"].notna()]
    if len(valid_vegas) > 0:
        corr = valid_vegas["bk_line"].corr(valid_vegas["vegas_line"])
        mae = (valid_vegas["bk_line"] - valid_vegas["vegas_line"]).abs().mean()
        
        print(f"\nVegas Line Comparison:")
        print(f"  Correlation: {corr:.3f}")
        print(f"  Mean Absolute Difference: {mae:.2f} points")
        print(f"  Games with Vegas line: {len(valid_vegas)}")

## 5. Edge Analysis

Identify games where our model differs most from Vegas.

In [ ]:
# Find biggest disagreements with Vegas
if "vegas_line" in predictions.columns:
    edge_analysis = predictions[["game_id", "away_team", "home_team", "bk_line", "vegas_line"]].copy()
    edge_analysis["difference"] = (edge_analysis["vegas_line"] - edge_analysis["bk_line"]).abs()
    edge_analysis = edge_analysis.dropna(subset=["vegas_line"])
    edge_analysis = edge_analysis.sort_values("difference", ascending=False)
    
    print("\n=== Top 5 Biggest Disagreements with Vegas ===")
    print(edge_analysis.head().to_string(index=False))

## 6. Export Predictions

Save predictions to CSV for further analysis.

In [ ]:
# Export to CSV
output_path = project_root / "predictions" / "bk_v1_0_week_11_2025.csv"
output_path.parent.mkdir(exist_ok=True)
predictions.to_csv(output_path, index=False)
print(f"\nPredictions saved to: {output_path}")

## Next Steps

**For v1.0:**
- Backtest on historical data (weeks 1-10)
- Calibrate weights based on historical performance
- Compare accuracy to Vegas closing lines

**For v1.1+:**
- Add QB adjustments (injury impacts)
- Incorporate rest days and travel
- Add weather adjustments for outdoor games
- Consider divisional game factors